In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
import random
import scipy
import pickle
import numpy as np

In [ ]:
import sys
sys.path.append('/mydata/watres/quentin/PinballRT/')
from UQ.UQ import UQ

In [ ]:
import numpy as np

def crps_from_quantiles(quantiles, tau, y):
    """
    Compute CRPS from predicted quantiles.
    
    Parameters:
    - quantiles: List or numpy array of predicted quantiles.
    - tau: List or numpy array of corresponding quantile levels (e.g., [0.1, 0.2, ..., 0.9]).
    - y: The observed value.
    
    Returns:
    - crps: The computed CRPS score.
    """
    quantiles = np.array(quantiles)
    tau = np.array(tau)
    
    # Compute the absolute errors
    errors = np.abs(quantiles - y)
    
    # Compute the weights as differences in tau levels
    weights = np.diff(np.insert(tau, 0, 0))  # Add 0 at the beginning for correct differences
    
    # Compute weighted sum
    crps = np.sum(weights * errors)
    
    return crps


def compute_crps(sample2predset, y_test, ls_q):
    res = 0
    for i, ytesti in enumerate(y_test):
        res += crps_from_quantiles([sample2predset[iq][i] for iq in range(len(ls_q))], ls_q, ytesti) / len(y_test)
    return res

In [ ]:
#ls_q = [0.1*i for i in range(1,10)]
ls_q = [0.05 * i for i in range(1, 20)]
#datasets = ['abalone',  'gas_turbine', 'combined_cycle_power_plant', 'red_wine', 'white_wine']
datasets = ['abalone',  'gas_turbine', 'combined_cycle_power_plant', 'red_wine','white_wine']

nbites = 27
comple = ''
lsntrain = [1000]
dataset2SKLEARNerrors = {dataset : np.zeros((nbites,len(lsntrain))) for dataset in datasets}
dataset2QNNerrors = {dataset : np.zeros((nbites,len(lsntrain))) for dataset in datasets}
dataset2QRFerrors = {dataset : np.zeros((nbites,len(lsntrain))) for dataset in datasets}
dataset2PQRTerrors = {dataset : np.zeros((nbites,len(lsntrain))) for dataset in datasets}
dataset2PMQRTerrors = {dataset : np.zeros((nbites,len(lsntrain))) for dataset in datasets}
dataset2PMQRT01errors = {dataset : np.zeros((nbites,len(lsntrain))) for dataset in datasets}
dataset2PMQRT05errors = {dataset : np.zeros((nbites,len(lsntrain))) for dataset in datasets}
dataset2CRPSRTerrors = {dataset : np.zeros((nbites,len(lsntrain))) for dataset in datasets}
dataset2CRPSRTAVGerrors = {dataset : np.zeros((nbites,len(lsntrain))) for dataset in datasets}
dataset2CRPSRT200errors = {dataset : np.zeros((nbites,len(lsntrain))) for dataset in datasets}
dataset2CRPSRTAVG200errors = {dataset : np.zeros((nbites,len(lsntrain))) for dataset in datasets}

root = '/mydata/watres/quentin/PinballRT/EXPE_FINAL/'


for name_dataset in datasets:
    for ite in range(1,nbites):
        print('ITE ', ite)
        print('\n')
        for i_ntrain, ntrain in enumerate(lsntrain):
            print('Ntrain: ', ntrain)

            y_test = np.load('sklearn_linear_new/{0}_{1}_{2}_y_test.npy'.format(ntrain, name_dataset, ite))
            with open(root+'sklearn_linear/{0}_{1}_{2}_sample2predset.pkl'.format(ntrain, name_dataset, ite), 'rb') as handle:
                sample2predset = pickle.load(handle)

            dataset2SKLEARNerrors[name_dataset][ite,i_ntrain] = compute_crps(sample2predset, y_test, ls_q)



            with open(root+'qrf_new/{0}_{1}_{2}_sample2predset.pkl'.format(ntrain, name_dataset, ite), 'rb') as handle:
                sample2predset = pickle.load(handle)
            dataset2QRFerrors[name_dataset][ite,i_ntrain] = compute_crps(sample2predset, y_test, ls_q)

            with open(root+'crpsrt_new/{0}_{1}_{2}_sample2predset.pkl'.format(ntrain, name_dataset, ite), 'rb') as handle:
                sample2predset = pickle.load(handle) 
            dataset2CRPSRTerrors[name_dataset][ite,i_ntrain] = compute_crps(sample2predset, y_test, ls_q)


            with open(root+'pmqrt_01_new/{0}_{1}_{2}_sample2predset.pkl'.format(ntrain, name_dataset, ite), 'rb') as handle:
                sample2predset = pickle.load(handle) 
            dataset2PMQRT01errors[name_dataset][ite,i_ntrain] = compute_crps(sample2predset, y_test, ls_q)

            with open(root+'pmqrt_05_new/{0}_{1}_{2}_sample2predset.pkl'.format(ntrain, name_dataset, ite), 'rb') as handle:
                sample2predset = pickle.load(handle)
            dataset2PMQRT05errors[name_dataset][ite,i_ntrain] = compute_crps(sample2predset, y_test, ls_q)


In [ ]:
ls_q = np.array(ls_q)
for name_dataset in datasets:
    print(name_dataset)
    x = dataset2SKLEARNerrors[name_dataset][:,0]   
    print('SKLEARN', np.mean(x), np.std(x))
    x = dataset2QRFerrors[name_dataset][:,0]   
    print('QRF', np.mean(x), np.std(x))
    x = dataset2PMQRT01errors[name_dataset][:,0]   
    print('PMQRT01', np.mean(x), np.std(x))
    x = dataset2PMQRT05errors[name_dataset][:,0]
    print('PMQRT05', np.mean(x), np.std(x))
    x = dataset2PMQRT05errors[name_dataset][:,0]   
    print('CRPS-RT', np.mean(x), np.std(x))
    print('\n')